# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [42]:
%reset

import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [66]:
%reset

import networkx as nx


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


We now encode the demands into the graph.

In [5]:
usa = open('contiguous-usa.dat')

print usa.readlines()

['AL FL\n', 'AL GA\n', 'AL MS\n', 'AL TN\n', 'AR LA\n', 'AR MO\n', 'AR MS\n', 'AR OK\n', 'AR TN\n', 'AR TX\n', 'AZ CA\n', 'AZ NM\n', 'AZ NV\n', 'AZ UT\n', 'CA NV\n', 'CA OR\n', 'CO KS\n', 'CO NE\n', 'CO NM\n', 'CO OK\n', 'CO UT\n', 'CO WY\n', 'CT MA\n', 'CT NY\n', 'CT RI\n', 'DC MD\n', 'DC VA\n', 'DE MD\n', 'DE NJ\n', 'DE PA\n', 'FL GA\n', 'GA NC\n', 'GA SC\n', 'GA TN\n', 'IA IL\n', 'IA MN\n', 'IA MO\n', 'IA NE\n', 'IA SD\n', 'IA WI\n', 'ID MT\n', 'ID NV\n', 'ID OR\n', 'ID UT\n', 'ID WA\n', 'ID WY\n', 'IL IN\n', 'IL KY\n', 'IL MO\n', 'IL WI\n', 'IN KY\n', 'IN MI\n', 'IN OH\n', 'KS MO\n', 'KS NE\n', 'KS OK\n', 'KY MO\n', 'KY OH\n', 'KY TN\n', 'KY VA\n', 'KY WV\n', 'LA MS\n', 'LA TX\n', 'MA NH\n', 'MA NY\n', 'MA RI\n', 'MA VT\n', 'MD PA\n', 'MD VA\n', 'MD WV\n', 'ME NH\n', 'MI OH\n', 'MI WI\n', 'MN ND\n', 'MN SD\n', 'MN WI\n', 'MO NE\n', 'MO OK\n', 'MO TN\n', 'MS TN\n', 'MT ND\n', 'MT SD\n', 'MT WY\n', 'NC SC\n', 'NC TN\n', 'NC VA\n', 'ND SD\n', 'NE SD\n', 'NE WY\n', 'NH VT\n', 'NJ NY\n'

In [10]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [11]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [67]:
def flow_with_demands(graph):

    #Calculating the total flow from sources and sinks respectively
    flow_value_s=0
    flow_value_t=0

    for nodes in graph.nodes():
        if graph.node[nodes]['demand']<0:
            flow_value_s=flow_value_s+graph.node[nodes]['demand']
        elif graph.node[nodes]['demand']>0:
            flow_value_t=flow_value_t+graph.node[nodes]['demand']

    #Condition that flow from source should be same as that into the sink
    if flow_value_s!=-flow_value_t:
        raise Exception('NetworkXUnfeasible')

    #Adding meta source and meta sink. Adding the edge capacity as the demand of node they are connected to
    for state in graph.nodes():
        if graph.node[state]['demand']<0:
            graph.add_edge('S',state)
            graph.edge['S'][state]['capacity'] = -graph.node[state]['demand']
        if graph.node[state]['demand']>0:
            graph.add_edge(state,'T')
            graph.edge[state]['T']['capacity'] = graph.node[state]['demand']

    #adding default demand value for S and T so that all nodes have demand paramete(Not really required)
    graph.node['S']['demand']=0
    graph.node['T']['demand']=0

    #Calculating maxflow
    flow_value, flow = nx.maximum_flow(graph, 'S', 'T')
    
    #max flow in the network should be equal to desired flow into sink or desired flow from source
    if flow_value!=flow_value_t:
        raise Exception('NetworkXUnfeasible')
    
    #deleting S and T from the flow dictionary
    del flow['S']
    del flow['T']
    for k in flow.keys():
        for k1 in flow[k].keys():
            if k1=='T':
                del flow[k][k1]
    
    #removing nodes S and T from G
    graph.remove_node('S')
    graph.remove_node('T')
    return flow

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [68]:
def divergence(flow):
    
    #Empty flows dict
    flows={}
    
    #Calculating flow into and out of a node and hence the total flow through the node
    for k in flow.keys():
        outflow=0
        for k1 in flow[k].keys():
            outflow=outflow+flow[k][k1]
        inflow=0
        for j in flow.keys():
            if j!=k:
                for k1 in flow[j].keys():
                    if k1==k:
                        inflow=inflow+flow[j][k]
        flows[k]=inflow-outflow                

    return flows

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [14]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True


In [74]:
#del graph1
graph1=nx.DiGraph()
graph1.add_node("A")
#graph1.add_node("B")
#graph1.add_node("C")
#graph1.add_edge("A","C")
#graph1.add_edge("A","B")
#graph1.edge["A"]["B"]['capacity']=1
graph1.node['A']['demand'] = 0
#graph1.node['B']['demand'] = -1
#graph1.node['C']['demand'] = 2

#for nodes in graph1.nodes():
#    print nodes
    #if graph1.node[nodes]['demand']<0:
    #    print graph1.node[nodes]['demand']
#print graph1.node["A"]["demand"]
flow_with_demands(graph1)
#graph1.nodes()

KeyError: 'S'